In [ ]:
import os

In [2]:
%pwd

'd:\\mlops\\datascienceproject\\research'

In [3]:
os.chdir("d:/mlops/datascienceproject/")
%pwd

'd:\\mlops\\datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

In [ ]:
@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [ ]:
from pathlib import Path
from src.DataScienceProject.constants import *
from src.DataScienceProject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_file_path=CONFIG_FILE_PATH,
        params_file_path=PARAMS_FILE_PATH,
        schema_file_path=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        # create artifact root
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([Path(config.root_dir)])

        data_ingestion_config = DataIngestionConfig(
            root_dir=Path(config.root_dir),
            source_URL=config.source_URL,
            local_data_file=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config


In [ ]:
import os
import urllib.request as request
from src.DataScienceProject import logger
import zipfile
## component-Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionConfig):
        self.config=config
    
    # Downloading the zip file
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)



In [ ]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-12-11 22:10:39,733: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-12-11 22:10:39,735: INFO: common: YAML file: params.yaml loaded successfully]
[2025-12-11 22:10:39,737: INFO: common: YAML file: schema.yaml loaded successfully]
[2025-12-11 22:10:39,739: INFO: common: created directory at: artifacts]
[2025-12-11 22:10:39,741: INFO: common: created directory at: artifacts\data_ingestion]
[2025-12-11 22:10:39,742: INFO: 3582574864: Downloading data from https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv to artifacts\data_ingestion\data.csv]
[2025-12-11 22:11:01,515: ERROR: 3582574864: Failed to download data: HTTPSConnectionPool(host='archive.ics.uci.edu', port=443): Read timed out. (read timeout=20)]


ReadTimeout: HTTPSConnectionPool(host='archive.ics.uci.edu', port=443): Read timed out. (read timeout=20)